### In this notebook, we will implement the scenarios for the Shimizu paper

The scenarios construct a lower dimensional mixing matrix `B` on $ \{3, 5, 10, 20, 100 \} $ variables. The exogenous error terms $ \varepsilon $ are expoentials $ \in [0.5, 0.8] \cup [1.2, 2.0] $ of normal distributions to achieve non-Gaussianity. The SEM reads $$ X = BX + \varepsilon. $$
We now construct the content for the parcs graph description file `graph_description.yml`

#  Code comparison
The LiNGAM implementation is taken from https://github.com/cdt15/lingam. The corresponding docs can be found here https://lingam.readthedocs.io/en/latest/.

# Adjacency Matrix Frobenius Norm Analysis

In [1]:
import lingam
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

### Create and save the datasets
We save the datasets to a specific data folder

In [17]:
from parcs.cdag.graph_objects import Graph
from parcs.graph_builder.parsers import graph_file_parser
import numpy as np
np.random.seed(2021)
for run in range(500):
    print(run)
    for attack in ["graph", "graph_correction", "graph_p=1"]:
        nodes, edges = graph_file_parser(attack + ".yml")
        samples = Graph(nodes=nodes, edges=edges).sample(size=1000)
        samples = samples.drop(columns = [colname for colname in samples.columns if colname[0].islower() and colname != "b" and colname != "sb"])
        samples.to_csv('./../data/' + attack + '/' + str(run) + '.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Let DirectLiNGAM run and evaluate
The evaluation of choice is the Frobenius norm, which is the 2-norm for matrices. We compare the original adjacency matrix against the estimated one.

In [18]:
np.random.seed(2023)

def m(samples):
    b = np.zeros((5,5))
    b[1,0] = np.sign(samples["sb"][0]) * samples["b"][0]
    b[2,0] = np.sign(samples["sb"][1]) * samples["b"][1]
    b[2,1] = np.sign(samples["sb"][2]) * samples["b"][2]
    b[3,0] = np.sign(samples["sb"][3]) * samples["b"][3]
    b[3,1] = np.sign(samples["sb"][4]) * samples["b"][4]
    b[3,2] = np.sign(samples["sb"][5]) * samples["b"][5]
    b[4,0] = np.sign(samples["sb"][6]) * samples["b"][6]
    b[4,1] = np.sign(samples["sb"][7]) * samples["b"][7]
    b[4,2] = np.sign(samples["sb"][8]) * samples["b"][8]
    b[4,3] = np.sign(samples["sb"][9]) * samples["b"][9]
    return b

runs = 500

d = np.zeros((runs, 3))
strength = np.zeros((runs, 3))
for a in range(3):
    attack = ["graph", "graph_correction", "graph_p=1"][a]
    for run in range(runs):
        print(attack, run)
        samples = pd.read_csv('./../data/' + attack + '/' + str(run) + '.csv')
        model = lingam.DirectLiNGAM()
        perm = np.random.permutation(5) + 4
        model.fit(samples.iloc[:,perm])
        d[run, a] = np.linalg.norm(np.array(model.adjacency_matrix_)[np.argsort(perm),:][:,np.argsort(perm)]  - m(samples))
        strength[run, a] = samples["P"][0]
        
pd.DataFrame(d).to_csv('./../data/d.csv')
pd.DataFrame(strength).to_csv('./../data/strength.csv')

graph 0
graph 1
graph 2
graph 3
graph 4
graph 5
graph 6
graph 7
graph 8
graph 9
graph 10
graph 11
graph 12
graph 13
graph 14
graph 15
graph 16
graph 17
graph 18
graph 19
graph 20
graph 21
graph 22
graph 23
graph 24
graph 25
graph 26
graph 27
graph 28
graph 29
graph 30
graph 31
graph 32
graph 33
graph 34
graph 35
graph 36
graph 37
graph 38
graph 39
graph 40
graph 41
graph 42
graph 43
graph 44
graph 45
graph 46
graph 47
graph 48
graph 49
graph 50
graph 51
graph 52
graph 53
graph 54
graph 55
graph 56
graph 57
graph 58
graph 59
graph 60
graph 61
graph 62
graph 63
graph 64
graph 65
graph 66
graph 67
graph 68
graph 69
graph 70
graph 71
graph 72
graph 73
graph 74
graph 75
graph 76
graph 77
graph 78
graph 79
graph 80
graph 81
graph 82
graph 83
graph 84
graph 85
graph 86
graph 87
graph 88
graph 89
graph 90
graph 91
graph 92
graph 93
graph 94
graph 95
graph 96
graph 97
graph 98
graph 99
graph 100
graph 101
graph 102
graph 103
graph 104
graph 105
graph 106
graph 107
graph 108
graph 109
graph 110


graph_correction 171
graph_correction 172
graph_correction 173
graph_correction 174
graph_correction 175
graph_correction 176
graph_correction 177
graph_correction 178
graph_correction 179
graph_correction 180
graph_correction 181
graph_correction 182
graph_correction 183
graph_correction 184
graph_correction 185
graph_correction 186
graph_correction 187
graph_correction 188
graph_correction 189
graph_correction 190
graph_correction 191
graph_correction 192
graph_correction 193
graph_correction 194
graph_correction 195
graph_correction 196
graph_correction 197
graph_correction 198
graph_correction 199
graph_correction 200
graph_correction 201
graph_correction 202
graph_correction 203
graph_correction 204
graph_correction 205
graph_correction 206
graph_correction 207
graph_correction 208
graph_correction 209
graph_correction 210
graph_correction 211
graph_correction 212
graph_correction 213
graph_correction 214
graph_correction 215
graph_correction 216
graph_correction 217
graph_correct

graph_p=1 109
graph_p=1 110
graph_p=1 111
graph_p=1 112
graph_p=1 113
graph_p=1 114
graph_p=1 115
graph_p=1 116
graph_p=1 117
graph_p=1 118
graph_p=1 119
graph_p=1 120
graph_p=1 121
graph_p=1 122
graph_p=1 123
graph_p=1 124
graph_p=1 125
graph_p=1 126
graph_p=1 127
graph_p=1 128
graph_p=1 129
graph_p=1 130
graph_p=1 131
graph_p=1 132
graph_p=1 133
graph_p=1 134
graph_p=1 135
graph_p=1 136
graph_p=1 137
graph_p=1 138
graph_p=1 139
graph_p=1 140
graph_p=1 141
graph_p=1 142
graph_p=1 143
graph_p=1 144
graph_p=1 145
graph_p=1 146
graph_p=1 147
graph_p=1 148
graph_p=1 149
graph_p=1 150
graph_p=1 151
graph_p=1 152
graph_p=1 153
graph_p=1 154
graph_p=1 155
graph_p=1 156
graph_p=1 157
graph_p=1 158
graph_p=1 159
graph_p=1 160
graph_p=1 161
graph_p=1 162
graph_p=1 163
graph_p=1 164
graph_p=1 165
graph_p=1 166
graph_p=1 167
graph_p=1 168
graph_p=1 169
graph_p=1 170
graph_p=1 171
graph_p=1 172
graph_p=1 173
graph_p=1 174
graph_p=1 175
graph_p=1 176
graph_p=1 177
graph_p=1 178
graph_p=1 179
graph_

### Alternative evaluation method:
Instead of measuring the entries of the adjacency matrix, we could also measure the distance between the estimated to the true causal order with the following function. Its output gives the number of necessary neighboring swaps to transform the first permutation into the second.
```def dist(a, d):
    if len(a) == 1:
        return d
    i_max = np.argmax(a)
    l = len(a)
    return dist(np.delete(a, i_max) - np.ones(l - 1), d + l - i_max - 1)```

# Plotting is carried out with R
The code can be found in a designated code_R folder.